In [1]:
import requests
import pandas as pd
import json
from legcop import LegiScan

legiscan_api_key='c8d73b2173b9881d495a8a05d59d860d'
legis = LegiScan(legiscan_api_key)

API Example

In [3]:
state='ca'
ca_sessions = legis.get_session_list(state=state)
print(ca_sessions)
start_most_recent_session = ca_sessions[0]['year_start']

datasetlist = legis.get_dataset_list(state=state,
                                     year=start_most_recent_session)

access_key = datasetlist[0]['access_key']
session_id =  datasetlist[0]['session_id']
dataset = legis.get_dataset(session_id=session_id, access_key=access_key)
assert dataset['status']=='OK'
readable_dataset = legis.recode_zipfile(dataset)
namelist = readable_dataset.namelist()
print(namelist)
item=namelist[0]
content = readable_dataset.read(item)
content = content.decode('UTF-8')
content = json.loads(content)

for key, value in content['bill'].items():
    print('{}: {}'.format(key, value))

[{'session_id': 2016, 'state_id': 5, 'year_start': 2023, 'year_end': 2024, 'prefile': 0, 'sine_die': 0, 'prior': 0, 'special': 0, 'session_tag': 'Regular Session', 'session_title': '2023-2024 Regular Session', 'session_name': '2023-2024 Session', 'dataset_hash': '89d69e51e88506408d80690952568b18', 'session_hash': '89d69e51e88506408d80690952568b18', 'name': '2023-2024 Regular Session'}, {'session_id': 1791, 'state_id': 5, 'year_start': 2021, 'year_end': 2022, 'prefile': 0, 'sine_die': 1, 'prior': 1, 'special': 0, 'session_tag': 'Regular Session', 'session_title': '2021-2022 Regular Session', 'session_name': '2021-2022 Session', 'dataset_hash': '48ae3222e08e6cd730ef7c818d467561', 'session_hash': '48ae3222e08e6cd730ef7c818d467561', 'name': '2021-2022 Regular Session'}, {'session_id': 1624, 'state_id': 5, 'year_start': 2019, 'year_end': 2020, 'prefile': 0, 'sine_die': 1, 'prior': 1, 'special': 0, 'session_tag': 'Regular Session', 'session_title': '2019-2020 Regular Session', 'session_name'

## CA Bill Master List

In [43]:
# Pull in full master list of CA bills
ca_masterlist = legis.get_master_list(state='ca')
df = pd.json_normalize(ca_masterlist)

In [44]:
# Data cleanup
df = df.drop(['prefile', 'sine_die', 'prior', 'special'], axis=1)
df['session_id'] = df['session_id'].fillna(method='ffill')
df['state_id'] = df['state_id'].fillna(method='ffill')
df['year_start'] = df['year_start'].fillna(method='ffill')
df['year_end'] = df['year_end'].fillna(method='ffill')
df['session_tag'] = df['session_tag'].fillna(method='ffill')
df['session_title'] = df['session_title'].fillna(method='ffill')
df['session_name'] = df['session_name'].fillna(method='ffill')
df['status_date'] = pd.to_datetime(df['status_date'])
df['last_action_date'] = pd.to_datetime(df['last_action_date'])
df = df.drop(index=0)
df = df.sort_values(by='last_action_date', ascending=False).reset_index(drop=True)
df.head(5)

Getting all bill titles

In [60]:
unique_bill_title = df['title'].unique()

for i in range(0, len(unique_bill_title)):
    print(unique_bill_title[i])

Personal income tax: California Senior Citizen Advocacy Voluntary Tax Contribution Fund.
Rehabilitative program providers.
Contractors State License Board: regulation of contractors.
Political Reform Act of 1974: Candidate statements.
Social media platforms: controlled substances: order to remove.
Vehicles: catalytic converters.
Alzheimer’s disease.
Venture capital companies: reporting.
Hazardous materials: enforcement: county counsel.
Cannabis provisional licenses: local equity applicants.
Hotel and private residence rental reservations: cancellation: refunds.
Sierra Nevada Conservancy: Sierra Nevada Region: subregions: climate resilience and equity.
Office of Wildfire Technology Research and Development: report on new technologies.
Surplus land disposal: violations: Orange County.
Special education: nonpublic, nonsectarian schools or agencies: change in certification status: parental notification.
Nominations: tax return disclosures: candidates for Governor.
Transportation: zero-emis

In [61]:
# We can possibly categorize the bills by groups if they include ":". This is just an idea off first glance, not all bills follow this format